<a href="https://colab.research.google.com/github/caio10012/Estudo_Python/blob/main/MLP/Otimizacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/valmirf/redes_neurais_pos.git

Cloning into 'redes_neurais_pos'...
remote: Enumerating objects: 120, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 120 (delta 11), reused 1 (delta 0), pack-reused 90 (from 1)
Receiving objects: 100% (120/120), 16.66 MiB | 27.21 MiB/s, done.
Resolving deltas: 100% (35/35), done.


##Multipayer Perceptron (MLP)

Rede Neural baseado no algoritmo de gradiente descendente.  
Os gradientes são calculados usando backpropagation.

Para mais detalhes, ver os capitulos 13 a 16 do livro no site:

http://deeplearningbook.com.br/

In [2]:
import random
import numpy as np

A entrada é uma lista (`sizes`) contém o número de neurônios nas respectivas camadas da rede. Por exemplo, se a lista for [2, 3, 1] então será uma rede de três camadas, com o primeira camada contendo 2 neurônios, a segunda camada 3 neurônios, e a terceira camada 1 neurônio. Os bias e pesos para a rede são inicializados aleatoriamente, usando uma distribuição Gaussiana com média 0 e variância 1. Note que a primeira camada é assumida como uma camada de entrada, e por convenção não definimos nenhum bias para esses neurônios, pois os bias são usados na computação das saídas das camadas posteriores.


In [14]:
# Classe Network
class Network(object):

    def __init__(self, sizes):
        self.num_layers = len(sizes)  #número de neurônios em cada camada
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]] #limiar
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])] #pesos

    def feedforward(self, x):
        """Retorna a saída da rede z se `x` for entrada."""
        for b, w in zip(self.biases, self.weights):
            x = sigmoid(np.dot(w, x)+b) #net = (∑wx+b)
        return x

    def SGD(self, training_data, epochs, mini_batch_size, 𝜂, test_data=None):
        """Treinar a rede neural usando o algoritmo mini batch com gradiente descendente.
         A entrada é uma lista de tuplas
         `(x, y)` representando as entradas de treinamento e as
         saídas. Os outros parâmetros não opcionais são
         auto-explicativos. Se `test_data` for fornecido, então a
         rede será avaliada em relação aos dados do teste após cada
         época e progresso parcial impresso. Isso é útil para
         acompanhar o progresso, mas retarda as coisas substancialmente."""

        #dataset de treino
        training_data = list(training_data)
        n = len(training_data)

        #dataset de teste
        if test_data:
            test_data = list(test_data)
            n_test = len(test_data)

        for j in range(epochs):
            random.shuffle(training_data)
            #técnica que realiza o treinamento por lotes
            #mini_batch_size = tamanho do lote
            mini_batches = [training_data[k:k+mini_batch_size] for k in range(0, n, mini_batch_size)]

            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, 𝜂)

            if test_data:
                acc = self.evaluate(test_data)
                print("Epoch {} : {} / {} = {}%".format(j,acc,n_test,(acc*100)/n_test));

            else:
                print("Epoch {} finalizada".format(j))

    def update_mini_batch(self, mini_batch, 𝜂):
        """Atualiza os pesos e limiares da rede aplicando
         a descida do gradiente usando backpropagation para um único mini lote.
         O `mini_batch` é uma lista de tuplas `(x, y)`, e `a` é a taxa de aprendizado."""

        #inicializa matriz com derivadas de pesos e limiares
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        nabla_b = [np.zeros(b.shape) for b in self.biases]

        for x, y in mini_batch:
            #resultado dos deltas do backpropagation sem a multiplicação da taxa de aprendizagem
            #soma os deltas do minibatch
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]

        #atualiza pesos e limiares (𝜂*𝛿*f’(net)*𝑥)
        self.weights = [w-(𝜂/len(mini_batch))*nw for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(𝜂/len(mini_batch))*nb for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """Retorna uma tupla `(nabla_b, nabla_w)` representando o
         gradiente para a função de custo J_x. `nabla_b` e
         `nabla_w` são listas de camadas de matrizes numpy, semelhantes
         a `self.biases` e `self.weights`."""
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        nabla_b = [np.zeros(b.shape) for b in self.biases]

        # Feedforward
        activation = x

        # Lista para armazenar todas as saídas dos neurônios (z), camada por camada
        activations = [x]

        # Lista para armazenar todos os vetores net, camada por camada
        nets = []

        for b, w in zip(self.biases, self.weights):
            net = np.dot(w, activation)+b
            nets.append(net)
            activation = sigmoid(net) #z = valor de saída do neurônio
            activations.append(activation)

        # Backward pass

        #última camada -(u-z)f'(net)
        delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(nets[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose()) #(𝑦−𝑧)*f’(net)*𝑥

        # l = 1 significa a última camada de neurônios, l = 2 é a penúltima e assim por diante.
        for l in range(2, self.num_layers):
            net = nets[-l]
            zs = sigmoid_prime(net)
            #delta da camada intermediaria. Note que utiliza o delta calculado anteriormente
            delta = np.dot(self.weights[-l+1].transpose(), delta) * zs
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose()) #∑(𝛿𝑤)f’(net)𝑥
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Retorna o número de entradas de teste para as quais a rede neural
         produz o resultado correto. Note que a saída da rede neural
         é considerada o índice de qualquer que seja
         neurônio na camada final que tenha a maior ativação."""

        test_results = [(np.argmax(self.feedforward(x)), y) for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Retorna o vetor das derivadas parciais."""
        return (output_activations-y)

# Função de Ativação Sigmóide
def sigmoid(net):
    return 1.0/(1.0+np.exp(-net))

# Função para retornar as derivadas da função Sigmóide
def sigmoid_prime(z):
    return sigmoid(z)*(1-sigmoid(z))


Como exemplo, essa mesma rede será executada na base de dados MNIST. O codigo abaixo carrega a base de dados.

In [15]:
# Carregar o dataset MNIST

# Imports
import pickle
import gzip
import numpy as np

def load_data():
    f = gzip.open('redes_neurais_pos/MLP/mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = pickle.load(f, encoding="latin1")
    f.close()
    return (training_data, validation_data, test_data)

def load_data_wrapper():
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = zip(training_inputs, training_results)
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = zip(validation_inputs, va_d[1])
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = zip(test_inputs, te_d[1])
    return (training_data, validation_data, test_data)

def vectorized_result(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e


#Executa a rede neural

Parâmetros de rede:
         2º param é contagem de épocas
         3º param é tamanho do lote
         4º param é a taxa de aprendizado (𝜂)




In [16]:
training_data, validation_data, test_data = load_data_wrapper()
training_data = list(training_data)

#arquitetura da rede
arquitecture = [784, 30, 10]
mlp = Network(arquitecture)
mlp.SGD(training_data, 30, 32, 0.3, test_data=test_data)




Epoch 0 : 4713 / 10000 = 47.13%
Epoch 1 : 5313 / 10000 = 53.13%
Epoch 2 : 6189 / 10000 = 61.89%
Epoch 3 : 6597 / 10000 = 65.97%
Epoch 4 : 6811 / 10000 = 68.11%
Epoch 5 : 6943 / 10000 = 69.43%
Epoch 6 : 7027 / 10000 = 70.27%
Epoch 7 : 7067 / 10000 = 70.67%
Epoch 8 : 7111 / 10000 = 71.11%
Epoch 9 : 7131 / 10000 = 71.31%
Epoch 10 : 7145 / 10000 = 71.45%
Epoch 11 : 7174 / 10000 = 71.74%
Epoch 12 : 7195 / 10000 = 71.95%
Epoch 13 : 7219 / 10000 = 72.19%
Epoch 14 : 7238 / 10000 = 72.38%
Epoch 15 : 7260 / 10000 = 72.6%
Epoch 16 : 7273 / 10000 = 72.73%
Epoch 17 : 7294 / 10000 = 72.94%
Epoch 18 : 7314 / 10000 = 73.14%
Epoch 19 : 7317 / 10000 = 73.17%
Epoch 20 : 7328 / 10000 = 73.28%
Epoch 21 : 7347 / 10000 = 73.47%
Epoch 22 : 7358 / 10000 = 73.58%
Epoch 23 : 7360 / 10000 = 73.6%
Epoch 24 : 7381 / 10000 = 73.81%
Epoch 25 : 7384 / 10000 = 73.84%
Epoch 26 : 7386 / 10000 = 73.86%
Epoch 27 : 7395 / 10000 = 73.95%
Epoch 28 : 7395 / 10000 = 73.95%
Epoch 29 : 7405 / 10000 = 74.05%


In [21]:
import numpy as np
import pickle
import gzip

# Funções do professor
def load_data():
    f = gzip.open('redes_neurais_pos/MLP/mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = pickle.load(f, encoding="latin1")
    f.close()
    return (training_data, validation_data, test_data)

def load_data_wrapper():
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = list(zip(training_inputs, training_results))
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = list(zip(validation_inputs, va_d[1]))
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = list(zip(test_inputs, te_d[1]))
    return (training_data, validation_data, test_data)

def vectorized_result(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

# Classe da rede neural
class NeuralNetwork:
    def __init__(self, sizes, regularization=None, lambda_=0.001, beta=0.9):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.weights = [np.random.randn(y, x) * np.sqrt(2 / x) for x, y in zip(sizes[:-1], sizes[1:])]
        self.biases = [np.zeros((y, 1)) for y in sizes[1:]]
        self.regularization = regularization
        self.lambda_ = lambda_
        self.beta = beta
        self.velocity_w = [np.zeros(w.shape) for w in self.weights]
        self.velocity_b = [np.zeros(b.shape) for b in self.biases]

    def feedforward(self, x):
        for b, w in zip(self.biases, self.weights):
            x = self.sigmoid(np.dot(w, x) + b)
        return x

    def update_mini_batch(self, mini_batch, eta):
        m = len(mini_batch)
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]

        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb + dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw + dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]

        if self.regularization == "L1":
            self.weights = [
                w - (eta / m) * nw - eta * self.lambda_ * np.sign(w) / m for w, nw in zip(self.weights, nabla_w)
            ]
        elif self.regularization == "L2":
            self.weights = [
                w - (eta / m) * nw - eta * self.lambda_ * w / m for w, nw in zip(self.weights, nabla_w)
            ]
        else:
            self.weights = [w - (eta / m) * nw for w, nw in zip(self.weights, nabla_w)]

        self.biases = [b - (eta / m) * nb for b, nb in zip(self.biases, nabla_b)]
        self.velocity_w = [self.beta * vw + (1 - self.beta) * nw for vw, nw in zip(self.velocity_w, nabla_w)]
        self.velocity_b = [self.beta * vb + (1 - self.beta) * nb for vb, nb in zip(self.velocity_b, nabla_b)]
        self.biases = [b - eta * vb for b, vb in zip(self.biases, self.velocity_b)]

    def backprop(self, x, y):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]

        activation = x
        activations = [x]
        zs = []

        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation) + b
            zs.append(z)
            activation = self.sigmoid(z)
            activations.append(activation)

        delta = self.cost_derivative(activations[-1], y) * self.sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].T)

        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = self.sigmoid_prime(z)
            delta = np.dot(self.weights[-l + 1].T, delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l - 1].T)

        return nabla_b, nabla_w

    def evaluate(self, test_data):
        test_results = [(np.argmax(self.feedforward(x)), y) for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        return output_activations - y

    @staticmethod
    def sigmoid(z):
        return 1.0 / (1.0 + np.exp(-z))

    @staticmethod
    def sigmoid_prime(z):
        return NeuralNetwork.sigmoid(z) * (1 - NeuralNetwork.sigmoid(z))

def train_network(configurations):
    training_data, _, test_data = load_data_wrapper()
    results = {"L1": [], "L2": [], "momento": []}

    for i, config in enumerate(configurations):
        print(f"\nTreinando configuração {i + 1}: {config}")
        for reg in ["L1", "L2", "momento"]:
            print(f"  Regularização: {reg}")
            net = NeuralNetwork(config, regularization=reg if reg != "momento" else None)

            for epoch in range(10):  # Número de épocas ajustável
                np.random.shuffle(training_data)
                mini_batches = [training_data[k:k + 10] for k in range(0, len(training_data), 10)]
                for mini_batch in mini_batches:
                    net.update_mini_batch(mini_batch, eta=0.1)  # Taxa de aprendizagem ajustada

                # Avaliar a acurácia após cada época
                accuracy = net.evaluate(test_data) / len(test_data)
                print(f"    Época {epoch + 1}: {accuracy * 100:.2f}% de acurácia")

            # Salvar o resultado final para essa configuração e regularização
            final_accuracy = net.evaluate(test_data) / len(test_data)
            results[reg].append(final_accuracy)
            print(f"  Acurácia final com {reg}: {final_accuracy * 100:.2f}%")
    return results

# Configurações de arquitetura
configurations = [[784, 32, 32, 10], [784, 16, 16, 10], [784, 32, 10]]
results = train_network(configurations)

# Mostrar resultados
for reg, acc in results.items():
    print(f"\n{reg} Results:")
    for i, res in enumerate(acc):
        print(f"Configuration {i + 1}: {res * 100:.2f}%")



Treinando configuração 1: [784, 32, 32, 10]
  Regularização: L1
    Época 1: 68.08% de acurácia
    Época 2: 84.13% de acurácia
    Época 3: 87.87% de acurácia
    Época 4: 89.55% de acurácia
    Época 5: 90.60% de acurácia
    Época 6: 90.84% de acurácia
    Época 7: 91.33% de acurácia
    Época 8: 91.63% de acurácia
    Época 9: 91.64% de acurácia
    Época 10: 91.96% de acurácia
  Acurácia final com L1: 91.96%
  Regularização: L2
    Época 1: 79.07% de acurácia
    Época 2: 87.89% de acurácia
    Época 3: 90.21% de acurácia
    Época 4: 91.12% de acurácia
    Época 5: 91.53% de acurácia
    Época 6: 92.31% de acurácia
    Época 7: 92.61% de acurácia
    Época 8: 92.94% de acurácia
    Época 9: 93.26% de acurácia
    Época 10: 93.59% de acurácia
  Acurácia final com L2: 93.59%
  Regularização: momento
    Época 1: 71.01% de acurácia
    Época 2: 88.91% de acurácia
    Época 3: 90.57% de acurácia
    Época 4: 91.32% de acurácia
    Época 5: 92.24% de acurácia
    Época 6: 92.61% de a

##**Mini-Projeto**

A partir dos melhores resultados obtidos no projeto anterior, execute 3 configurações pra cada questão a seguir:

1) Implementar as regularizações L1 e L2

2) Implementar o Momento

3) Comparar os experimentos e explicar o porque de cada resultado? Qual foi a melhor regularização? Por quê? O Momento melhorou os resultados? Por quê?


L1 -> Desempenho estável em todas as configurações, com a Configuração 1 ligeiramente superior. Isso sugere que a arquitetura mais profunda (com mais camadas intermediárias) consegue capturar padrões mais complexos mesmo com L1.

 L2 -> foi eficaz porque permitiu que o modelo aprendesse de forma equilibrada e controlada, penalizando pesos grandes de forma suave, o que ajudou a reduzir a complexidade do modelo e melhorar a sua capacidade de generalização. Isso é importante por contado do  risco de overfitting ser alto por conta dos grande números de parâmetros.

 Melhor regularização -> L2 foi a melhor técnica de regularização entre as testadas. ela proporcionou uma penalização mais equilibrada e suave dos pesos, o que resultou em uma melhor capacidade de generalização do modelo. A L1, por ser mais agressiva, pode ter eliminado pesos importantes demais, o que afetou a performance geral.

  Momento -> foi a técnica mais eficaz, com desempenho consistentemente superior. Isso ocorre porque o momento acelera a convergência acumulando os gradientes anteriores, o que permite um aprendizado mais rápido e eficiente. Isso resulta em melhor performance, especialmente em configurações mais complexas.

Data de Entrega: 14/10/2020
     
      
Complete a Tabela abaixo com os resultados

\begin{array}{|c|c|c|c|}\hline\\ \\
  Configuracao & [784, 32, 32, 10] & [784, 16,16, 10] & [784, 32, 10] \\ \hline
L1 & 91.96 & 91.24 &  91.90 \\ \hline
L2  & 93.59 & 92.25 &  93.39  \\ \hline
Momento  & 94.05&93.02 &94.01 \\ \hline
\end{array}



\\
Data de Entrega: 17/10/2020
